In [ ]:
import requests
import pandas as pd
from tqdm import tqdm
import time
import os

# Configuration pour Haïti
latitude = 18.9712
longitude = -72.2852
start_year = 2010
end_year = 2024

def get_climate_data(year):
    """Récupère les données climatiques pour une année donnée"""
    try:
        url = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}"
        url += f"&start_date={year}-01-01&end_date={year}-12-31"
        url += "&daily=temperature_2m_max,temperature_2m_min,precipitation_sum,relative_humidity_2m_mean"
        url += "&timezone=auto"

        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        return pd.DataFrame({
            'date': data['daily']['time'],
            'annee': year,
            'temp_max': data['daily']['temperature_2m_max'],
            'temp_min': data['daily']['temperature_2m_min'],
            'precipitation': data['daily']['precipitation_sum'],
            'humidite': data['daily']['relative_humidity_2m_mean']
        })

    except Exception as e:
        print(f"Erreur pour {year}: {str(e)}")
        return None

# Collecte des données avec progression
print(f"Extraction des données climatiques pour Haïti ({start_year}-{end_year})...")
all_data = []

for year in tqdm(range(start_year, end_year + 1), desc="Progression"):
    df_year = get_climate_data(year)
    if df_year is not None:
        all_data.append(df_year)
    time.sleep(1)  # Pause pour éviter le rate limiting

if all_data:
    # Fusion des données
    df_final = pd.concat(all_data, ignore_index=True)

    # Conversion des dates
    df_final['date'] = pd.to_datetime(df_final['date'])

    # Création d'un dossier local de sauvegarde s’il n'existe pas
    output_dir = "../data"
    os.makedirs(output_dir, exist_ok=True)

    # Sauvegarde dans un fichier CSV local
    file_path = os.path.join(output_dir, f'climat_haiti_{start_year}_{end_year}.csv')
    df_final.to_csv(file_path, index=False)

    print(f"\nDonnées sauvegardées avec succès : {file_path}")
    print(f"Nombre total de jours : {len(df_final)}")
    print("\nAperçu des données :")
    print(df_final.head())
else:
    print("Aucune donnée n'a pu être récupérée.")
